In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

In [32]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
	percentage = 0.75
	data_loader = DataLoader(percentage, True, param['random_seed'])
	use_only_card = param['use_only_CARD']
	size_of_dataset = param['size_of_dataset']
	del param['size_of_dataset']
	del param['use_only_CARD']

	train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)
	folds = data_loader.create_cv_folds(train_df, test_df)

	for train_df, test_df in folds:
		print("run")
		clf = ClusterAndPredict(**param, train_df=train_df)
		clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
		# Print best parameters
		best_estimator = clf
		score = best_estimator.score([], [])
		print(best_estimator.get_all_performance_metrics())
		object_output = best_estimator.get_all_performance_metrics()
		cluster_df = object_output['cluster_df']
		cluster_dfs.append(cluster_df)
		output = {
			'percentage': percentage,
			'score': score,
			'accuracy': best_estimator.get_accuracy(),
			'was_supervised_umap_used': best_estimator.get_was_supervised(),
			'metrics': best_estimator.get_all_performance_metrics(),
			'size_of_dataset': size_of_dataset,
			'use_only_CARD': use_only_card,
		}
		# Prepend the value 'param' to the keys in params
		for key, value in param.items():
			local_key = 'params.' + key
			output[local_key] = value
		results.append(output)

All data sources are being used
Number of experiments to run:  1
All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64
run
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  3622
Getting embeddings for batch  75  out of  3622
Getting embeddings for batch  150  out of  3622
Getting embeddings for batch  225  out of  3622
Getting embeddings for batch  300  out of  3622
Getting embeddings for batch  375  out of  3622
Getting embeddings for batch  450  out of  3622
Getting embeddings for batch  525  out of  3622
Getting embeddings for batch  600  out of  3622
Getting embeddings for batch  675  out of  3622
Getting embeddings for batch  750  out of  3622
Getting embeddings for batch  825  out of  3622
Getting embeddings for batch  900  out of  3622
Getting embeddings for batch  975  out of  3622
Getting embeddings for batch  1050  out of  3622
Getting embeddings for batc

/home/codespace/.local/lib/python3.10/site-packages/pynndescent/pynndescent_.py:962: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


Running hdbscan...
breaking further...
279
279
279
279
284
286
288
290
292
292
294
294
296
296
296
296
296
296
296
296
296
296
296
298
298
298
298
298
301
301
305
305
305
305
307
309
309
309
309
309
309
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
cluster  predict  veracity
-1       False     3          1290
                   1           934
         True     -1          2126
 1       False     1             5
 2       False     3             3
                              ... 
 310     False     1             1
                   3             1
         True     -1             3
 311     False     3             7
         True     -1             4
Name: count, Length: 492, dtype: int64
breaking further...
311
311
315
315
315
315
318
318
318
320
320
320
320
320
320
cluster  predict  veracity
-1       False     3          1350
                   1           988
         True     -1          2224
 1

KeyboardInterrupt: 

In [33]:
results_df = pd.json_normalize(results)
# Get current date and time
from datetime import datetime
now = datetime.now()
results_df
results_df.to_csv(f'all_data_results_${str(now)}.csv', index=False)

In [42]:
data = results_df['metrics.cluster_df'].iloc[0]
from datetime import datetime
now = datetime.now()
data
data.to_csv(f'all_data_results_cluster_df${str(now)}.csv', index=False)